In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
sns.set()

In [ ]:
data = pd.read_csv('../input/video-game-speedrun-and-tas-world-record-times/f_dataset.csv')
data.drop('Unnamed: 0',axis=1,inplace=True)
data

In [ ]:
data.columns

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='median')
imp.fit(data['time'].values.reshape(-1,1))

In [ ]:
data['time'] = imp.transform(data['time'].values.reshape(-1,1))

In [ ]:
features = ['time','main_platform','main_genre','engine','developer','publisher','emulator','Unpopular', 'Somehow_Popular', 'Popular',
       'Very_Popular', 'Young', 'Somehow_Old', 'Old', 'Very_Old', 'Short',
       'Somehow_Long']
label = ['TAS_time']

In [ ]:
X = pd.get_dummies(data[features])
y = data[label]

In [ ]:
X.to_csv('test.csv')

In [ ]:
features = X.columns
features

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
X_train.reset_index(inplace=True,drop=True)
X_test.reset_index(inplace=True,drop=True)
y_train.reset_index(inplace=True,drop=True)
y_test.reset_index(inplace=True,drop=True)

In [ ]:
import statsmodels.api as sm
X_train_sm= sm.add_constant(X_train)

In [ ]:
p_values = []
for feature in features:
    ls=sm.OLS(y_train,X_train_sm[feature]).fit()
    print(ls.summary())
    p_values.append(ls.pvalues)

In [ ]:
t = []
for i in range(len(p_values)):
    if p_values[i].values < 0.2 :
        print(p_values[i])
        t.append(p_values[i].index)

In [ ]:
t

In [ ]:
significative_features = ['time', 
 'Unpopular', 
 'Somehow_Popular', 
 'Popular', 
 'Very_Popular', 
 'Young', 
 'Very_Old',
 'Short',
 'Somehow_Long', 
 'main_platform_Game Boy Advance', 
 'main_platform_Game Boy Color', 
 'main_platform_GameCube', 
 'main_platform_Nintendo 3DS Virtual Console', 
 'main_platform_Nintendo 64', 
 'main_platform_Nintendo DS', 
 'main_platform_Nintendo Entertainment System', 
 'main_platform_PlayStation', 
 'main_platform_PlayStation 2', 
 'main_platform_Sega Genesis', 
 'main_platform_Sega Saturn', 
 'main_platform_Super Nintendo', 
 'main_platform_Wii U', 
 'main_platform_Wii Virtual Console', 
 'main_genre_Action', 
 'main_genre_Action Adventure', 
 'main_genre_Action-adventure', 
 'main_genre_Capcom', 
 'main_genre_JRPG', 
 'main_genre_Platform', 
 'main_genre_Platformer', 
 'main_genre_RPG', 
 'main_genre_Role-playing Game', 
 'main_genre_Strategy', 
 'engine_A.L.I.V.E', 
 'developer_Capcom', 
 'developer_Capcom Production Studio 3', 
 'developer_Chunsoft', 
 'developer_DeerForce', 
 'developer_Game Freak', 
 'developer_Insomniac Games', 
 'developer_Intelligent Systems', 
 'developer_Konami Computer Entertainment Osaka', 
 'developer_NCS', 
 'developer_Nintendo', 
 'developer_Nintendo R&D1', 
 'developer_Rare', 
 'developer_Skawo', 
 'developer_Square', 
 'developer_Sucker Punch Productions', 
 'publisher_Capcom', 
 'publisher_Enix', 
 'publisher_GT Interactive', 
 'publisher_Hudson Soft', 
 'publisher_Nintendo', 
 'publisher_SEGA', 
 'publisher_Skawo', 
 'publisher_Sony Computer Entertainment', 
 'publisher_Square', 
 'publisher_Treco', 
 'publisher_Ubisoft', 
 'emulator_BizHawk', 
 'emulator_DeSmuME', 
 'emulator_Dolphin', 
 'emulator_Mupen64', 
 'emulator_PCSX', 
 'emulator_PSXjin', 
 'emulator_snes9x']

In [ ]:
ls=sm.OLS(y_train,X_train_sm[significative_features]).fit()
print(ls.summary())

In [ ]:
def MeanAbsoluteError(Y_test,y_test):
    n = len(Y_test)
    num = 0
    for i in range(n):
        num += abs(y_test[i]-Y_test.loc[i,label])

    return num[label].values[0]/n
def RMSE(Y_test,y_test):
    import math
    n = len(Y_test)
    num = 0
    for i in range(n):
        num += (y_test[i]-Y_test.loc[i,label])*(y_test[i]-Y_test.loc[i,label])

    return (math.sqrt(num[label].values/n))

def bias(Y_test,y_test):
    n = len(Y_test)
    num = 0
    for i in range(n):
        num += y_test[i]-Y_test.loc[i,label]

    return num[label].values[0]/n

In [ ]:
from sklearn import linear_model
model1 = LinearRegression().fit(X_train['time'].values.reshape(-1,1), y_train)
print("Training test R² : "+str( model1.score( X_train['time'].values.reshape(-1,1), y_train ) ) )
print("Test set Mean Absolue Error : "+str(MeanAbsoluteError(y_test,model1.predict(X_test['time'].values.reshape(-1,1)))))
print( "Test set RMSE : "+ str(RMSE(y_test,model1.predict(X_test['time'].values.reshape(-1,1)))) + " ("+str( RMSE(y_test,model1.predict(X_test['time'].values.reshape(-1,1))) * 100 / y_test[label].values.mean() ) +"% of the mean)")
print('Standard deviation of test set : '+str(y_test[label].values.std())+" ("+str(y_test[label].values.std()*100/y_test[label].values.mean())+"% of the mean)")


In [ ]:
ychapo_test_model1_array = model1.predict(X_test['time'].values.reshape(-1,1))
ychapo_model1 = []
for i in range(len(ychapo_test_model1_array)):
    ychapo_model1.append(ychapo_test_model1_array[i][0])

In [ ]:
sns.relplot(x=X_test['time'],y=ychapo_model1)

In [ ]:
sns.relplot(x=X_test['time'],y=y_test['TAS_time'])

In [ ]:
model1.coef_

In [ ]:
custom_features = ['time', 
 'Unpopular', 
 'Somehow_Popular', 
 'Popular', 
 'Very_Popular', 
 'Young', 
 'Very_Old',
 'Short',
 'Somehow_Long']

In [ ]:
from sklearn import linear_model
model2 = LinearRegression().fit(X_train[significative_features], y_train)
print("Training test R² : "+str( model2.score( X_train[significative_features], y_train ) ) )
print("Test set Mean Absolue Error : "+str(MeanAbsoluteError(y_test,model2.predict(X_test[significative_features]))))
print( "Test set RMSE : "+ str(RMSE(y_test,model2.predict(X_test[significative_features]))) + " ("+str( RMSE(y_test,model2.predict(X_test[significative_features])) * 100 / y_test[label].values.mean() ) +"% of the mean)")

In [ ]:
ychapo_test_model2_array = model2.predict(X_test[significative_features])
ychapo_model2 = []
for i in range(len(ychapo_test_model2_array)):
    ychapo_model2.append(ychapo_test_model2_array[i][0])

In [ ]:
sns.relplot(x=X_test['time'],y=ychapo_model2)

In [ ]:
sns.relplot(x=X_test['time'],y=y_test['TAS_time'])

In [ ]:
alphas = 10**np.linspace(10,-2,100)*0.5
model3 = linear_model.RidgeCV(alphas=alphas,cv=3).fit(X_train[significative_features], y_train)
print("Training test R² : "+str( model3.score( X_train[significative_features], y_train ) ) )
print("Test set Mean Absolue Error : "+str(MeanAbsoluteError(y_test,model3.predict(X_test[significative_features]))))
print( "Test set RMSE : "+ str(RMSE(y_test,model3.predict(X_test[significative_features]))) + " ("+str( RMSE(y_test,model3.predict(X_test[significative_features])) * 100 / y_test[label].values.mean() ) +"% of the mean)")

In [ ]:
ychapo_test_model3_array = model3.predict(X_test[significative_features])
ychapo_model3 = []
for i in range(len(ychapo_test_model3_array)):
    ychapo_model3.append(ychapo_test_model3_array[i][0])

In [ ]:
sns.relplot(x=X_test['time'],y=ychapo_model3)

In [ ]:
alphas = 10**np.linspace(10,-2,100)*0.5
model4 = linear_model.RidgeCV(alphas=alphas,cv=3).fit(X_train, y_train)
print("Training test R² : "+str( model4.score( X_train, y_train ) ) )
print("Test set Mean Absolue Error : "+str(MeanAbsoluteError(y_test,model4.predict(X_test))))
print( "Test set RMSE : "+ str(RMSE(y_test,model4.predict(X_test))) + " ("+str( RMSE(y_test,model4.predict(X_test)) * 100 / y_test[label].values.mean() ) +"% of the mean)")

In [ ]:
model5 = linear_model.LassoCV(alphas=alphas).fit(X_train, np.array(y_train).ravel())
print("Training test R² : "+str( model5.score( X_train, np.array(y_train).ravel() ) ) )
print("Biais on test set : "+str(bias( y_test,model8.predict(X_test) ) ) )
print("Test set Mean Absolue Error : "+str(MeanAbsoluteError(y_test,model5.predict(X_test))))
print( "Test set RMSE : "+ str(RMSE(y_test,model5.predict(X_test))) + " ("+str( RMSE(y_test,model5.predict(X_test)) * 100 / y_test[label].values.mean() ) +"% of the mean)")

In [ ]:
ychapo_model5 = model5.predict(X_test)

In [ ]:
sns.relplot(x=X_test['time'],y=ychapo_model5)

In [ ]:
model6 = linear_model.ElasticNetCV(l1_ratio=np.arange(0.01,1,0.01)).fit(X_train, np.array(y_train).ravel())
print("Training test R² : "+str( model6.score( X_train, np.array(y_train).ravel() ) ) )
print("Test set Mean Absolue Error : "+str(MeanAbsoluteError(y_test,model6.predict(X_test))))
print( "Test set RMSE : "+ str(RMSE(y_test,model6.predict(X_test))) + " ("+str( RMSE(y_test,model6.predict(X_test)) * 100 / y_test[label].values.mean() ) +"% of the mean)")

In [ ]:
from sklearn import linear_model
model7 = LinearRegression().fit(X_train[custom_features], y_train)
print("Training test R² : "+str( model7.score( X_train[custom_features], y_train ) ) )
print("Test set Mean Absolue Error : "+str(MeanAbsoluteError(y_test,model7.predict(X_test[custom_features]))))
print( "Test set RMSE : "+ str(RMSE(y_test,model7.predict(X_test[custom_features]))) + " ("+str( RMSE(y_test,model7.predict(X_test[custom_features])) * 100 / y_test[label].values.mean() ) +"% of the mean)")

In [ ]:
ychapo_test_model7_array = model7.predict(X_test[custom_features])
ychapo_model7 = []
for i in range(len(ychapo_test_model7_array)):
    ychapo_model7.append(ychapo_test_model7_array[i][0])

In [ ]:
sns.relplot(x=X_test['time'],y=ychapo_model7)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

parameters = {'n_estimators':np.arange(70,90,1)}
grid = GridSearchCV(RandomForestRegressor(), parameters)
model8 = grid.fit(X_train,np.array(y_train).ravel())

In [ ]:
grid.best_params_

In [ ]:
print("Training test R² : "+str( model8.score( X_train, np.array(y_train).ravel() ) ) )
print("Biais on test set : "+str(bias( y_test,model8.predict(X_test) ) ) )
print("Test set Mean Absolue Error : "+str(MeanAbsoluteError(y_test,model8.predict(X_test))))
print( "Test set RMSE : "+ str(RMSE(y_test,model8.predict(X_test))) + " ("+str( RMSE(y_test,model8.predict(X_test)) * 100 / y_test[label].values.mean() ) +"% of the mean)")

In [ ]:
parameters = {'n_estimators':np.arange(70,90,1),'criterion':('mse','mae'),'max_features':('auto','log2','sqrt')}
grid = GridSearchCV(RandomForestRegressor(), parameters,scoring='neg_mean_absolute_error',cv=3)
model9 = grid.fit(X_train,np.array(y_train).ravel())

In [ ]:
grid.best_params_

In [ ]:
print("Training test Neg Mean Absolute Error : "+str( model9.score( X_train, np.array(y_train).ravel() ) ) )
print("Biais on test set : "+str(bias( y_test,model9.predict(X_test) ) ) )
print("Test set Mean Absolue Error : "+str(MeanAbsoluteError(y_test,model9.predict(X_test))))
print( "Test set RMSE : "+ str(RMSE(y_test,model9.predict(X_test))) + " ("+str( RMSE(y_test,model9.predict(X_test)) * 100 / y_test[label].values.mean() ) +"% of the mean)")

In [ ]:
sns.relplot(x=X_test['time'],y=model8.predict(X_test))

In [ ]:
test_times = []
for time in y_test['TAS_time'].values:
    test_times.append(datetime.timedelta(seconds=time))

predicted_times = []
for time in model9.predict(X_test):
    predicted_times.append(datetime.timedelta(seconds=time))

diff = []
for i in range(len(test_times)):
    diff.append(abs(test_times[i]-predicted_times[i]))

In [ ]:
df = pd.DataFrame({'test_times':test_times,'predicted_times':predicted_times,'diff':diff})
df

In [ ]:
df.describe()

In [ ]:
df[df['diff'] == df['diff'].max()]